***Important notes***:

Before any running please upload ExplainWD.csv in the session storage. To do so, you can use the Files section in the left side of this page.

How to run the code:

In the first cell, initialize the important variables:

dimension: This dataset has 4 dimensions, hence the dimension value should be kept to 4
random_state: We tried three different random_state for sampling data: 200, 345, and 546. It sets 200 as a defualt. You can change it in this cell if you want.

1. Run the entire cells for "Installs and utils", "Data Loading and seeds", and "Preparing data" sections

2. Run one of the following models from Metods section:

      1. ERNIE
      2. XLNET
      3. PsychBERT
      4. ClinicalBERT
      5. MentalBERT
      6. BERT
      7. RoBERTa

3. Run the entire cells in the MCC results section

Note that if you are going to run the code again for another method, you must run it from the first section again.

You are done!

In [ ]:
dimension= 4
# We tried three different random_state for sampling data: 200, 345, and 546.
# It sets 200 as a defualt. You can change it in this cell if you want.
rand_state = 200


# Do not change the following.
MAX_LEN = 64
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE= 1e-05
flag=False

#Installs and utils

In this section, requirements like numpy, pandas libraries are imported. Also, the pretrained models that we are using in this code are downloaded in this section.

Action required: you may need to log in into hugginface in the second cell, for usage of MentalBERT

In [ ]:
!pip install transformers
!pip install SentencePiece
!pip install tensorflow_addons
!pip install bertviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      S

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [ ]:
import pandas as pd
import numpy as np
from bertviz import model_view
from transformers import utils
utils.logging.set_verbosity_error()
import shutil, sys

In [ ]:
import torch

In [ ]:
from transformers import AutoModel, AutoTokenizer
from transformers import XLNetModel, XLNetTokenizer

#Data Loading

**Action required:**

Before runing this cell, you will need to move ExplainWD.csv file into the path that this code is. On Google colab, you can upload MultiWD.csv file into Session Storage.

In [ ]:
# Read the ExplainWD dataset.
data_new=pd.read_csv('ExplainWD.csv')

#This portion involves generating one-hot encoded vectors of the dimensions

If a post has dimension as aspect 3, then the one-hot vector would be <0 0 1 0>

If a post has dimension as spect 1, then the one-hot vector would be <1 0 0 0>

In [ ]:
data=pd.DataFrame()
data['Text']=data_new['Text']
data['Aspect']=data_new['Aspect']
data['Aspect1']=data_new['Aspect']
data['Aspect2']=data_new['Aspect']
data['Aspect3']=data_new['Aspect']
data['Aspect4']=data_new['Aspect']
data['Explanations']=data_new['Explanations']

In [ ]:
for i in range(1,5):
  if i!=1:
    data['Aspect1']=data['Aspect1'].replace(i,0)

In [ ]:
for i in range(1,5):
  if i==2:
    data['Aspect2']=data['Aspect2'].replace(i,1)
  else:
    data['Aspect2']=data['Aspect2'].replace(i,0)

In [ ]:
for i in range(1,5):
  if i==3:
    data['Aspect3']=data['Aspect3'].replace(i,1)
  else:
    data['Aspect3']=data['Aspect3'].replace(i,0)

In [ ]:
for i in range(1,5):
  if i==4:
    data['Aspect4']=data['Aspect4'].replace(i,1)
  else:
    data['Aspect4']=data['Aspect4'].replace(i,0)

In [ ]:
data

,Text,Aspect,Aspect1,Aspect2,Aspect3,Aspect4,Explanations
0,Once it happens there is no going back and you...,4,0,0,0,1,empty void of darkness
1,Now i am filled with guilt and stuck feeling l...,4,0,0,0,1,filled with guilt and stuck feeling like crap
2,And i got home and the voices returned and kep...,4,0,0,0,1,piece of shit
3,"Before her I survived since shit, sexual assau...",1,1,0,0,0,sexual assaults
4,I feel like there are only three things that y...,1,1,0,0,0,"ugly, dumb, and useless"
...,...,...,...,...,...,...,...
3087,"I am mature student in, Toxic class mates, bas...",4,0,0,0,1,hate myself
3088,"i want to be dead, everyone hates me",4,0,0,0,1,everyone hates me
3089,I believe that I am selfish and feel because I...,4,0,0,0,1,selfish
3090,"i dont wanna die, i want to live to be happy t...",4,0,0,0,1,hate myself


In [ ]:

# Preparing the dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['Text']
        self.targets = self.df[target_List].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [ ]:
# Functions for saving and loading the model in the case the training
# is interrupted. In this case, we use these functions start training
# again from last check point.

def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
# setting up and running CUDA operations. In case it is not available.
# It changes to cpu.

device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [ ]:
#Sigmoid Loss Function set up.

def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
# train_model is the function to train the model on the training data.

def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf


  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs,_ = model(ids, mask, token_type_ids)
        #print(outputs.size())

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)

    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs,_ = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAverage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

#4-label

In [ ]:
data=data.astype({'Aspect1':'float', 'Aspect2':'float', 'Aspect3':'float',
       'Aspect4':'float'})

In [ ]:
target_List = ['Aspect1','Aspect2','Aspect3','Aspect4']

In [ ]:
# Sampling data for training set and validation set
test_size = 0.2
val_df = data.sample(frac=test_size, random_state=rand_state).reset_index (drop=True)
train_df = data.drop (val_df.index).reset_index (drop=True)

#Method

In these secions, we have provided the code for training models: ERNIE, XLNET, PsychBERT, ClinicalBERT, MentalBERT, BERT, and RoBERTa.

***You should not run all of the code in this section. Instead make sure you run one of them based on your choice.***

###ERNIE

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nghuyong/ernie-2.0-en')

In [ ]:
train_dataset=CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader (
train_dataset,
shuffle=True,
batch_size=TRAIN_BATCH_SIZE,
num_workers=0
)
val_data_loader = torch.utils.data.DataLoader (
valid_dataset,
shuffle=False,
batch_size=VALID_BATCH_SIZE,
num_workers=0
)

In [ ]:
class ERNIEClass(torch.nn.Module):
    def __init__(self):
        super(ERNIEClass, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained('nghuyong/ernie-2.0-en')
        self.ernie_model = AutoModel.from_pretrained('nghuyong/ernie-2.0-en', output_attentions=True, return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 4)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output_dict = self.ernie_model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids,
            output_hidden_states=True,
            return_dict=True
        )
        last_hidden_state = output_dict.last_hidden_state
        attention_weights = output_dict.attentions
        output_dropout = self.dropout(last_hidden_state[:, -1, :])
        output = self.linear(output_dropout)
        return output, attention_weights

model = ERNIEClass()
model.to(device)

ERNIEClass(
  (ernie_model): ErnieModel(
    (embeddings): ErnieEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(4, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ErnieEncoder(
      (layer): ModuleList(
        (0-11): 12 x ErnieLayer(
          (attention): ErnieAttention(
            (self): ErnieSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ErnieSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, elem

In [ ]:
val_targets=[]
val_outputs=[]
val_list=[]

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/ckpt_path", "/best.pt")

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Average Training Loss: 0.004812 	Average Validation Loss: 0.017124
Validation loss decreased (inf --> 0.017124).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Average Training Loss: 0.003246 	Average Validation Loss: 0.014425
Validation loss decreased (0.017124 --> 0.014425).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epoch

In [ ]:
final_list=[]
evaluation=[]
for i in val_df['Text']:
  example = i
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output,output_with_attention = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      # final=[0 if i<0.4 else 1 for i in final_output[0]]
      # print("final",final)
      input_id_list = input_ids[0].tolist()
      tokens = tokenizer.convert_ids_to_tokens(input_id_list)
      # raise KeyBoardInteruupt
      #model_view(output_with_attention,tokens)

      final_list.append(final_output[0])

In [ ]:
for i in range(len(val_df)):
  val_list.append(val_df[target_List][i:i+1].values.tolist()[0])

In [ ]:
def finalLabels(predicted_list,val_list):
  for i in range(len(predicted_list)):
    indices=np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))]
    # argsort()[:-1][:n]
    # print(predicted_list,np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))])
    for j in range(len(predicted_list[i])):
      if j in indices:
        predicted_list[i][j]=1.0
      else:
        predicted_list[i][j]=0.0
  return predicted_list

In [ ]:
finalLabels(final_list,val_list)

[[0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0,

In [ ]:
from sklearn.metrics import classification_report
label_names = target_List
report=classification_report(val_list, final_list,target_names=label_names)

In [ ]:
print(report)

              precision    recall  f1-score   support

     Aspect1       0.90      0.55      0.68       144
     Aspect2       0.83      0.96      0.89       110
     Aspect3       0.94      0.99      0.96       233
     Aspect4       0.77      0.92      0.84       131

   micro avg       0.87      0.87      0.87       618
   macro avg       0.86      0.86      0.84       618
weighted avg       0.87      0.87      0.86       618
 samples avg       0.87      0.87      0.87       618



In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(val_list, final_list)

print("Accuracy: {}".format(accuracy))

Accuracy: 0.8673139158576052


###XLNET

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
flag=True

In [ ]:
train_dataset=CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader (
train_dataset,
shuffle=True,
batch_size=TRAIN_BATCH_SIZE,
num_workers=0
)
val_data_loader = torch.utils.data.DataLoader (
valid_dataset,
shuffle=False,
batch_size=VALID_BATCH_SIZE,
num_workers=0
)

In [ ]:
class XLNetClass(torch.nn.Module):
    def __init__(self):
        super(XLNetClass, self).__init__()
        self.xlnet_model = XLNetModel.from_pretrained('xlnet-base-cased', output_attentions=True, return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 4)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output_dict = self.xlnet_model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids,
            output_hidden_states=True,
            return_dict=True
        )
        last_hidden_state = output_dict.last_hidden_state
        attention_weights = output_dict.attentions
        output_dropout = self.dropout(last_hidden_state[:, -1, :])
        output = self.linear(output_dropout)
        return output, attention_weights[-1]

model = XLNetClass()
model.to(device)

XLNetClass(
  (xlnet_model): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0-11): 12 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dropout): Dropout(p=0.3, inplace=False)
  (linear): Linear(in_features=768, out_features=4, bias=True)
)

In [ ]:
val_targets=[]
val_outputs=[]
val_list=[]

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/ckpt_path", "/best.pt")

############# Epoch 1: Training Start   #############


In [ ]:
final_list=[]
for i in val_df['Text']:
  example = i
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output,output_with_attention = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      # final=[0 if i<0.4 else 1 for i in final_output[0]]
      # print("final",final)
      final_list.append(final_output[0])

In [ ]:
val_list=[]

In [ ]:
for i in range(len(val_df)):
  val_list.append(val_df[target_List][i:i+1].values.tolist()[0])

In [ ]:
def finalLabels(predicted_list,val_list):
  for i in range(len(predicted_list)):
    indices=np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))]
    # argsort()[:-1][:n]
    # print(predicted_list,np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))])
    for j in range(len(predicted_list[i])):
      if j in indices:
        predicted_list[i][j]=1.0
      else:
        predicted_list[i][j]=0.0
  return predicted_list

In [ ]:
finalLabels(final_list,val_list)

In [ ]:
from sklearn.metrics import classification_report
label_names = target_List
print(classification_report(val_list, final_list,target_names=label_names))

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(val_list, final_list)

print("Accuracy: {}".format(accuracy))

###PsychBERT

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('mnaylor/psychbert-cased')

In [ ]:
train_dataset=CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader (
train_dataset,
shuffle=True,
batch_size=TRAIN_BATCH_SIZE,
num_workers=0
)
val_data_loader = torch.utils.data.DataLoader (
valid_dataset,
shuffle=False,
batch_size=VALID_BATCH_SIZE,
num_workers=0
)

In [ ]:
class PsychBERTClass(torch.nn.Module):
    def __init__(self):
        super(PsychBERTClass, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
        self.model = AutoModel.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 4)

    def forward(self, input_ids, attn_mask, seg_ids):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=seg_ids
        )
        output_with_attention = output
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output, output_with_attention


model = PsychBERTClass()
model.to(device)

PsychBERTClass(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
val_targets=[]
val_outputs=[]
val_list=[]

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/ckpt_path", "/best.pt")

In [ ]:
final_list=[]
for i in val_df['Text']:
  example = i
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output,output_with_attention = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      # final=[0 if i<0.4 else 1 for i in final_output[0]]
      # print("final",final)
      input_id_list = input_ids[0].tolist()
      tokens = tokenizer.convert_ids_to_tokens(input_id_list)
      # raise KeyBoardInteruupt
      #model_view(output_with_attention.attentions,tokens)

      final_list.append(final_output[0])

In [ ]:
for i in range(len(val_df)):
  val_list.append(val_df[target_List][i:i+1].values.tolist()[0])

In [ ]:
def finalLabels(predicted_list,val_list):
  for i in range(len(predicted_list)):
    indices=np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))]
    # argsort()[:-1][:n]
    # print(predicted_list,np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))])
    for j in range(len(predicted_list[i])):
      if j in indices:
        predicted_list[i][j]=1.0
      else:
        predicted_list[i][j]=0.0
  return predicted_list

In [ ]:
finalLabels(final_list,val_list)

[[0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0,

In [ ]:
from sklearn.metrics import classification_report
label_names = target_List
print(classification_report(val_list, final_list,target_names=label_names))

              precision    recall  f1-score   support

     Aspect1       0.00      0.00      0.00       161
     Aspect2       0.00      0.00      0.00       125
     Aspect3       0.36      1.00      0.53       221
     Aspect4       0.00      0.00      0.00       111

   micro avg       0.36      0.36      0.36       618
   macro avg       0.09      0.25      0.13       618
weighted avg       0.13      0.36      0.19       618
 samples avg       0.36      0.36      0.36       618



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(val_list, final_list)

print("Accuracy: {}".format(accuracy))

Accuracy: 0.35760517799352753


###ClinicalBERT

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')

In [ ]:
train_dataset=CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader (
train_dataset,
shuffle=True,
batch_size=TRAIN_BATCH_SIZE,
num_workers=0
)
val_data_loader = torch.utils.data.DataLoader (
valid_dataset,
shuffle=False,
batch_size=VALID_BATCH_SIZE,
num_workers=0
)

In [ ]:
class ClinicalBIGBERTClass(torch.nn.Module):
    def __init__(self):
        super(ClinicalBIGBERTClass, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
        self.model = AutoModel.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 5)

    def forward(self, input_ids, attn_mask, seg_ids):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=seg_ids
        )
        output_with_attention = output
        output_dropout = self.dropout(output.last_hidden_state[:, 0])
        output = self.linear(output_dropout)
        return output, output_with_attention



model = ClinicalBIGBERTClass()
model.to(device)

ClinicalBIGBERTClass(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
val_targets=[]
val_outputs=[]
val_list=[]

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/ckpt_path", "/best.pt")

In [ ]:
final_list=[]
for i in val_df['text']:
  example = i
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output,output_with_attention = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      # final=[0 if i<0.4 else 1 for i in final_output[0]]
      # print("final",final)
      input_id_list = input_ids[0].tolist()
      tokens = tokenizer.convert_ids_to_tokens(input_id_list)
      # raise KeyBoardInteruupt
      #model_view(output_with_attention.attentions,tokens)

      final_list.append(final_output[0])

In [ ]:
for i in range(len(val_df)):
  val_list.append(val_df[target_List][i:i+1].values.tolist()[0])

In [ ]:
def finalLabels(predicted_list,val_list):
  for i in range(len(predicted_list)):
    indices=np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))]
    # argsort()[:-1][:n]
    # print(predicted_list,np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))])
    for j in range(len(predicted_list[i])):
      if j in indices:
        predicted_list[i][j]=1.0
      else:
        predicted_list[i][j]=0.0
  return predicted_list

In [ ]:
finalLabels(final_list,val_list)

[[0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 1.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 1.0, 1.0, 1.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 1.0, 1.0],
 [0.0, 0.0, 0.0, 0.0

In [ ]:
from sklearn.metrics import classification_report
label_names = target_List
print(classification_report(val_list, final_list,target_names=label_names))

                        precision    recall  f1-score   support

              Physical       0.99      1.00      1.00       190
          Intellectual       0.98      1.00      0.99       127
                Social       1.00      1.00      1.00       440
            Vocational       1.00      0.98      0.99       115
Spiritual or Emotional       1.00      1.00      1.00       300

             micro avg       1.00      1.00      1.00      1172
             macro avg       1.00      1.00      1.00      1172
          weighted avg       1.00      1.00      1.00      1172
           samples avg       0.99      0.99      0.99      1172



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(val_list, final_list)

print("Accuracy: {}".format(accuracy))

Accuracy: 0.9939024390243902


###MentalBERT

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
tokenizer=AutoTokenizer.from_pretrained('mental/mental-bert-base-uncased',use_auth_token=True)

In [ ]:
train_dataset=CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader (
train_dataset,
shuffle=True,
batch_size=TRAIN_BATCH_SIZE,
num_workers=0
)
val_data_loader = torch.utils.data.DataLoader (
valid_dataset,
shuffle=False,
batch_size=VALID_BATCH_SIZE,
num_workers=0
)

In [ ]:
class MentalBERTClass(torch.nn.Module):
    def __init__(self):
        super(MentalBERTClass, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained('mental/mental-bert-base-uncased')
        self.model = AutoModel.from_pretrained('mental/mental-bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 3)

    def forward(self, input_ids, attn_mask, seg_ids):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=seg_ids
        )
        output_with_attention = output
        output_dropout = self.dropout(output.last_hidden_state[:, 0])
        output = self.linear(output_dropout)
        return output, output_with_attention



model = MentalBERTClass()
model.to(device)

MentalBERTClass(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
val_targets=[]
val_outputs=[]
val_list=[]

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/ckpt_path", "/best.pt")

In [ ]:
final_list=[]
for i in val_df['text']:
  example = i
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output,output_with_attention = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      # final=[0 if i<0.4 else 1 for i in final_output[0]]
      # print("final",final)
      input_id_list = input_ids[0].tolist()
      tokens = tokenizer.convert_ids_to_tokens(input_id_list)
      # raise KeyBoardInteruupt
      #model_view(output_with_attention.attentions,tokens)

      final_list.append(final_output[0])

In [ ]:
for i in range(len(val_df)):
  val_list.append(val_df[target_List][i:i+1].values.tolist()[0])

In [ ]:
def finalLabels(predicted_list,val_list):
  for i in range(len(predicted_list)):
    indices=np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))]
    # argsort()[:-1][:n]
    # print(predicted_list,np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))])
    for j in range(len(predicted_list[i])):
      if j in indices:
        predicted_list[i][j]=1.0
      else:
        predicted_list[i][j]=0.0
  return predicted_list

In [ ]:
finalLabels(final_list,val_list)

In [ ]:
from sklearn.metrics import classification_report
label_names = target_List
print(classification_report(val_list, final_list,target_names=label_names))

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(val_list, final_list)

print("Accuracy: {}".format(accuracy))

Accuracy: 0.9435975609756098


###BERT

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
train_dataset=CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader (
train_dataset,
shuffle=True,
batch_size=TRAIN_BATCH_SIZE,
num_workers=0
)
val_data_loader = torch.utils.data.DataLoader (
valid_dataset,
shuffle=False,
batch_size=VALID_BATCH_SIZE,
num_workers=0
)

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.model = AutoModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, dimension)

    def forward(self, input_ids, attn_mask, seg_ids):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=seg_ids
        )
        output_with_attention = output
        output_dropout = self.dropout(output.last_hidden_state[:, 0])
        output = self.linear(output_dropout)
        return output, output_with_attention



model = BERTClass()
model.to(device)

MentalBERTClass(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
val_targets=[]
val_outputs=[]
val_list=[]

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/ckpt_path", "/best.pt")

In [ ]:
final_list=[]
for i in val_df['Text']:
  example = i
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output,output_with_attention = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      # final=[0 if i<0.4 else 1 for i in final_output[0]]
      # print("final",final)
      input_id_list = input_ids[0].tolist()
      tokens = tokenizer.convert_ids_to_tokens(input_id_list)
      # raise KeyBoardInteruupt
      #model_view(output_with_attention.attentions,tokens)

      final_list.append(final_output[0])

In [ ]:
for i in range(len(val_df)):
  val_list.append(val_df[target_List][i:i+1].values.tolist()[0])

In [ ]:
def finalLabels(predicted_list,val_list):
  for i in range(len(predicted_list)):
    indices=np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))]
    # argsort()[:-1][:n]
    # print(predicted_list,np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))])
    for j in range(len(predicted_list[i])):
      if j in indices:
        predicted_list[i][j]=1.0
      else:
        predicted_list[i][j]=0.0
  return predicted_list

In [ ]:
finalLabels(final_list,val_list)

In [ ]:
from sklearn.metrics import classification_report
label_names = target_List
print(classification_report(val_list, final_list,target_names=label_names))

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(val_list, final_list)

print("Accuracy: {}".format(accuracy))

Accuracy: 0.9862804878048781


###roBERTa

In [ ]:
tokenizer=AutoTokenizer.from_pretrained("roberta-base")

In [ ]:
train_dataset=CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader (
train_dataset,
shuffle=True,
batch_size=TRAIN_BATCH_SIZE,
num_workers=0
)
val_data_loader = torch.utils.data.DataLoader (
valid_dataset,
shuffle=False,
batch_size=VALID_BATCH_SIZE,
num_workers=0
)

In [ ]:
class roBERTaClass(torch.nn.Module):
    def __init__(self):
        super(roBERTaClass, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained("roberta-base")
        self.model = AutoModel.from_pretrained("roberta-base", return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, dimension)

    def forward(self, input_ids, attn_mask, seg_ids):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=seg_ids
        )
        output_with_attention = output
        output_dropout = self.dropout(output.last_hidden_state[:, 0])
        output = self.linear(output_dropout)
        return output, output_with_attention



model = roBERTaClass()
model.to(device)

roBERTaClass(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [ ]:
val_targets=[]
val_outputs=[]
val_list=[]

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/ckpt_path", "/best.pt")

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Average Training Loss: 0.006429 	Average Validation Loss: 0.020203
Validation loss decreased (inf --> 0.020203).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Average Training Loss: 0.003849 	Average Validation Loss: 0.016978
Validation loss decreased (0.020203 --> 0.016978).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epoch

In [ ]:
import pickle
file_name = 'ExplainWD_model_'+'RoBERTa'+'_'+'SCE_'+str(dimension)+'_Dimension'+'.pkl'
pickle.dump(model, open(file_name, 'wb'))

In [ ]:
final_list=[]
for i in val_df['Text']:
  example = i
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output,output_with_attention = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      # print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
      # final=[0 if i<0.4 else 1 for i in final_output[0]]
      # print("final",final)
      input_id_list = input_ids[0].tolist()
      tokens = tokenizer.convert_ids_to_tokens(input_id_list)
      # raise KeyBoardInteruupt
      #model_view(output_with_attention.attentions,tokens)

      final_list.append(final_output[0])

In [ ]:
for i in range(len(val_df)):
  val_list.append(val_df[target_List][i:i+1].values.tolist()[0])

In [ ]:
def finalLabels(predicted_list,val_list):
  for i in range(len(predicted_list)):
    indices=np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))]
    # argsort()[:-1][:n]
    # print(predicted_list,np.array(predicted_list[i]).argsort()[::-1][:int(sum(val_list[i]))])
    for j in range(len(predicted_list[i])):
      if j in indices:
        predicted_list[i][j]=1.0
      else:
        predicted_list[i][j]=0.0
  return predicted_list

In [ ]:
finalLabels(final_list,val_list)

[[0.0, 0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0,

In [ ]:
from sklearn.metrics import classification_report
label_names = target_List
print(classification_report(val_list, final_list,target_names=label_names))

              precision    recall  f1-score   support

     Aspect1       0.94      0.41      0.57       144
     Aspect2       0.75      0.96      0.84       110
     Aspect3       0.89      0.98      0.93       233
     Aspect4       0.73      0.88      0.80       131

   micro avg       0.82      0.82      0.82       618
   macro avg       0.83      0.81      0.79       618
weighted avg       0.84      0.82      0.80       618
 samples avg       0.82      0.82      0.82       618



In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(val_list, final_list)

print("Accuracy: {}".format(accuracy))

Accuracy: 0.8220064724919094


#Calculation of SVD Ranking

In [ ]:
last_layer_attentions = []
for batch_idx, data in enumerate(train_data_loader):
    ids = data['input_ids'].to(device, dtype=torch.long)
    mask = data['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
    outputs, output_with_attention = model(ids, mask, token_type_ids)

    if flag==True:
        attentions = output_with_attention  # For XLNET
    elif flag==False:
        attentions = output_with_attention.attentions[0]

    for sample in attentions:
        last_layer_attentions.append((sample[11]).detach().cpu())  # Detach and move to CPU

    # Clear GPU memory
    del ids, mask, token_type_ids, outputs, output_with_attention
    torch.cuda.empty_cache()

In [ ]:
from numpy.linalg import svd
from numpy.linalg import matrix_rank

d=[item.detach().numpy() for item in last_layer_attentions]
U, S, VT = svd(d)

print('*********************************************')
# print("Experiment:", dimension)
print("SVD_ranking:", matrix_rank(S))

#MCC Results

In [ ]:
import tensorflow_addons as tfa
from sklearn.metrics import matthews_corrcoef

metric = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=dimension)
metric.update_state(val_list,final_list)
result = metric.result()
print(result.numpy())
print('target_List:', target_List)

val = pd.DataFrame(val_list, columns = target_List)
fin = pd.DataFrame(final_list, columns = target_List)

from sklearn.metrics import matthews_corrcoef
print(matthews_corrcoef(val["Aspect1"],fin["Aspect1"]))

from sklearn.metrics import matthews_corrcoef
print(matthews_corrcoef(val["Aspect2"],fin["Aspect2"]))

from sklearn.metrics import matthews_corrcoef
print(matthews_corrcoef(val["Aspect3"],fin["Aspect3"]))

from sklearn.metrics import matthews_corrcoef
print(matthews_corrcoef(val["Aspect4"],fin["Aspect4"]))


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


0.76686376
target_List: ['Aspect1', 'Aspect2', 'Aspect3', 'Aspect4']
0.5606668116943069
0.8155595689154229
0.8911952154711125
0.7397442650590393
